In [1]:
%pip install lxml polars


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: /Users/pletcher/code/writing/articles/2024-11-28_tragedy-dfs/.venv/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import polars as pl

df = pl.read_parquet("./greek-tragedy-by-line.parquet").with_columns(
    tokens=pl.col("text").str.split(" ")
)
df

In [41]:
with pl.Config(tbl_rows=-1):
    res = (
        df.filter(pl.col("title") != "Ichneutae")
        .group_by("dramatist", "title")
        .agg(pl.col("tokens").explode())
        .select(
            "dramatist",
            "title",
            n_types=pl.col("tokens").list.n_unique(),
            n_tokens=pl.col("tokens").list.len(),
            ttr=(pl.col("tokens").list.n_unique() / pl.col("tokens").list.len()),
        )
        .sort("ttr")
    )

    print(res)

    avgs = (
        res.group_by("dramatist")
        .agg(pl.col("ttr").mean().alias("avg_ttr"))
        .select("dramatist", "avg_ttr")
        .sort("avg_ttr")
    )

    print(avgs)

shape: (33, 5)
┌───────────┬──────────────────────┬─────────┬──────────┬──────────┐
│ dramatist ┆ title                ┆ n_types ┆ n_tokens ┆ ttr      │
│ ---       ┆ ---                  ┆ ---     ┆ ---      ┆ ---      │
│ str       ┆ str                  ┆ u32     ┆ u32      ┆ f64      │
╞═══════════╪══════════════════════╪═════════╪══════════╪══════════╡
│ Sophocles ┆ Οἰδίπους Τύραννος    ┆ 4465    ┆ 9609     ┆ 0.464669 │
│ Sophocles ┆ Οἰδίπους ἐπὶ Κολωνῷ  ┆ 5043    ┆ 10702    ┆ 0.47122  │
│ Sophocles ┆ Φιλοκτήτης           ┆ 4295    ┆ 8888     ┆ 0.483236 │
│ Sophocles ┆ Ἠλέκτρα              ┆ 4365    ┆ 8744     ┆ 0.499199 │
│ Euripides ┆ Ὀρέστης              ┆ 5066    ┆ 9867     ┆ 0.513429 │
│ Euripides ┆ Ἑλένη                ┆ 5038    ┆ 9790     ┆ 0.514607 │
│ Euripides ┆ Ἰφιγένεια ἐν Αὐλίδι  ┆ 4740    ┆ 9208     ┆ 0.51477  │
│ Euripides ┆ Ἴων                  ┆ 4836    ┆ 9224     ┆ 0.524284 │
│ Euripides ┆ Μήδεια               ┆ 4163    ┆ 7892     ┆ 0.527496 │
│ Euripides ┆ Ἡρακλ